<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+CTR

Merge the bibliographic datasets for 
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

Implementation:
The input pure o partially processed database with WOS-SCI-SCP and may be some UDEA entries from PTJ and Center information with additional data about the Full Name UDEA authors.

Addtionaly UDEA entries can be captured from:
1. A previous WOS-SCI-SCP-UDEA
2. A Data Base with a column with full names (FULL LAST NAMES NAMES, e.g VALDEZ GÚZMAN JUAN ALBERTO) and a list of author Aliases in WOS format (Lastname, Name, e.g Valdez-Gúzman, J.A.) with a list of registered affiliations. TODO: Test
3. The database from Puntaje (UDEA). 

In [2]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus > /dev/null

In [3]:
# Delete UDEA_columns and start from schratch
REBUILD=False
MERGE_WITH_TRAINED=False

## functions

In [4]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)

In [5]:
# %load wos_sci_scp_ptj_ctr.py

In [6]:
from wos_sci_scp_ptj_ctr import *

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [7]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
UDEA_WOS_SCI_SCP_PTJ.json=1OkVytKbxJwGvXZDkynkSoUDtkUOTaT4A

Overwriting drive.cfg


##  Load data bases

In [8]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

#### DEBUG: if False stop in UDEA_PTJ!!!!

if os.path.exists(UDEAjsonfile):
    UDEA=               pd.read_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)
else:    
    UDEA=drive_files.read_drive_json(UDEAjsonfile,compression='gzip').reset_index(drop=True)

In [726]:
RECOVER=True #False for test purposes
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
#Test purposes
#UDEAjsonfile='UDEA_WOS_SCI_SCP_PTJ.json'
if RECOVER:
    #Requieres latest wosplus!
    tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')# TODO CHANGE FOR LAST VERSION IN GOOGLE DRIVE
else:
    tmp=drive_files.load_biblio('UDEAtmp.json')
    #drive_files.load_biblio(
    #  'https://raw.githubusercontent.com/restrepo/medicion/master/cienciometria/data/UDEAtmp300.json'
    #    )#Test: 199+1=200 found
    
UDEA=drive_files.biblio['WOS'].reset_index(drop=True)
#DEBUG
#UDEA=UDEA.sample(300,replace=True).reset_index(drop=True) #Test: 77 found
#tmp=drive_files.load_biblio('Sample_WOS.xlsx')

In [84]:
if REBUILD:
    UDEA=clean_institutional_columns(UDEA,prefix='UDEA',Tipo='Tipo')
    UDEA['UDEA_authors']=None


In [727]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

WOS_SCP_UDEA:1043
SCI:2258
WOS:1541
SCP:2112
WOS_SCP:4777
WOS_SCI_SCP:557
SCI_SCP:1116
WOS_UDEA:343
SCI_SCP_UDEA:500
SCI_UDEA:634
SCP_UDEA:461
WOS_SCI_SCP_UDEA:211
WOS_SCI_UDEA:47
WOS_SCI:100


In [728]:
UDEA.shape

(15700, 181)

## Load trained old data 

### Merge WOS_SCP_SCI with trained data set PTJ_CTR

Merge requires split in DI and TI


15700 (15700, 152)
(7072, 169) (8628, 169)

In [730]:
SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

In [87]:
if MERGE_WITH_TRAINED:
    if os.path.exists('WOS_SCP_UDEA_SJR_SIU.xlsx'):
        SIU=pd.read_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
    else:    
        SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
        
    UDEA,SIU=fill_trained_data(UDEA,SIU)#TODO: Remnove SIU

In [88]:
if MERGE_WITH_TRAINED:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [89]:
if 'UDEA_autores' in UDEA.columns and UDEA[UDEA['UDEA_autores']==''].shape[0]:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s)

In [90]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 10311


# Puntaje

UDEA

In [91]:
qq=UDEA.copy()

In [92]:
drive_files.biblio['WOS']=qq
drive_files.biblio['WOS'].shape

(15700, 181)

In [93]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [94]:
pp= drive_files.biblio['UDEA'].copy()

In [95]:
drive_files.biblio['UDEA']=pp

In [96]:
df=merge_puntaje(drive_files)

(32581, 24)
va1 0 0
......................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5388 + 1 = 5389
va2 0 5388
......................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


7258 : 5388 + 0 = 5388
va3 0 5388
......................................................

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


va4 0 5388
7258 : 5388 + 0 = 5388
(1, 174) + (5388, 152) = 5389


In [97]:
#TODO: Check why not zero
if 'UDEA_autores' in df.columns:
    print(0,'=',df[df['UDEA_autores']==''].shape[0],'; found:',df['UDEA_autores'].dropna().shape[0])

0 = 0 ; found: 10312


In [98]:
#df['UDEA_autores'].apply(lambda s: pd.np.nan if type(s)==str and s=='' else s).dropna().shape

In [99]:
UDEA=df.copy()

In [100]:
UDEA.shape

(15700, 181)

## Fill C1 for not WOS entries in WOS format and extract  affiliation from C1

In [101]:
#Fill from SCI_C1
UDEA['C1']=SCI_C1_to_C1(UDEA)

In [102]:
#Fill from SCP_C1='SCP_Authors with affiliations
UDEA['C1']=SCP_Authors_with_affiliations_to_C1(UDEA)

In [103]:
UDEA[UDEA['C1'].isnull()].shape

(0, 181)

In [104]:
UDEA[UDEA.Tipo=='WOS'].reset_index(drop=True).C1.loc[0]

'[Bedoya Hernandez, Mauricio Hernando] Univ Antioquia, Educ, Medellin, Colombia.\n[Bedoya Hernandez, Mauricio Hernando] Univ Antioquia, Psicol, Medellin, Colombia.\n[Bedoya Hernandez, Mauricio Hernando] Univ Antioquia, Ciencias Sociales, Medellin, Colombia.\n[Bedoya Hernandez, Mauricio Hernando] Univ Antioquia, Medellin, Colombia.\n'

In [105]:
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

# Improve normalization: remove C1s with only affiliation (from Scielo)
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( 
    lambda x: [d for d in x if d.get('WOS_author').find(affil)==-1] if type(x)==list else x )

In [106]:
UDEA[UDEA.Tipo=='SCP'].reset_index(drop=True).loc[0].authors_WOS

[{'WOS_author': 'Mesa-Vanegas, A. M.',
  'affiliation': ['Grupo de Investigación Malaria, Sede de Investigación Universitaria(SIU), Facultad de Medicina, Univ Antioquia, Medellín, Colombia'],
  'i': 0},
 {'WOS_author': 'Cardona, F.',
  'affiliation': ['Grupo de Estudios Botánicos, Herbario Univ Antioquia, Medellín, Colombia'],
  'i': 1},
 {'WOS_author': 'Sáez-Vega, J. A.',
  'affiliation': ['Grupo Química de Plantas Colombianas, Instituto de Química, Univ Antioquia, Medellín, Colombia'],
  'i': 2},
 {'WOS_author': 'Trujillo, S. B.',
  'affiliation': ['Grupo de Investigación Malaria, Sede de Investigación Universitaria(SIU), Facultad de Medicina, Univ Antioquia, Medellín, Colombia'],
  'i': 3}]

## Prepare UDEA columns

In [107]:
#TODO: Remove from fill_trained_data(..)
if 'UDEA_autores' in UDEA.columns:
    UDEA['UDEA_autores']=UDEA['UDEA_autores'].apply(lambda s: re.sub('\s+',' ',s) if type(s)==str else s)
    UDEA['UDEA_authors']=UDEA['UDEA_autores'].apply(lambda s: s.split(';') if type(s)==str else s).apply(
                           lambda l: [{'full_name':y} for y in l ] if type(l)==list else l)

## Merge with official researcher list: PTJ

In [108]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [109]:
UPDATE_UDEA_authors_with_AU=True
if (UDEA['UDEA_authors'].dropna().shape[0] and 
    UPDATE_UDEA_authors_with_AU):
    kkn=UDEA.copy()
    kkn=update_institutional_authors(kkn,AU)
    print(kkn.shape,UDEA.shape)
    UDEA=kkn.copy()

0
1
2
3
4
5
6
7
8
9
10
11
(15700, 181) (15700, 181)


Quality check

In [110]:
key_contains_in_list_of_dictionaries(UDEA,'Restrepo, D',column='authors_WOS',key='WOS_author').loc[1:2]

1    [{'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.'], 'i': 0, 'WOS_author': 'Restrepo, Diego'}]
2       [{'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.'], 'i': 0, 'WOS_author': 'Restrepo, D.'}]
Name: authors_WOS, dtype: object

In [111]:
if UPDATE_UDEA_authors_with_AU:
    UDEA.to_json('UDEAtmp.json')
    RECOVER=False
    if RECOVER:
        UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Add `UDEA.authors_WOS` info* within `UDEA.UDEA_authors` data**
(\*) obtained from `UDEA.C1`

(\*\*) Obtained from [puntaje trained old UDEA data](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-trained-data-set) and the [official researcher list](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-official-researcher-list)

Obtain name parts and initials from full name in `UDEA_authors` dictionary and update `UDEA_authors` with them

In [112]:
import sys
if 'UDEA_authors' not in UDEA.columns and REBUILD==False:
    sys.exit('Make MERGE_WITH_TRAINED True and run again')

In [113]:
# Obtain spanish name parts from full name
dictupdatetmp=UDEA['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [114]:
kk=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=combinewos )

In [115]:
UDEA['UDEA_authors'].loc[0]

[{'INICIALES': 'D. S.',
  'NOMBRES': 'Del Socorro',
  'PRIMER APELLIDO': 'Lopez',
  'SEGUNDO APELLIDO': 'Gomez',
  'WOS_affiliation': ['Univ Antioquia, Fac Ciencias Econ, Bogota, Colombia.'],
  'WOS_author': ['Lopez Gomez, Maria del Socorro'],
  'full_name': 'LOPEZ GOMEZ MARIA DEL SOCORRO'}]

In [116]:
UDEA.to_json('UDEAtmp.json')

### Load output restuls of previous Cell runs

In [117]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Build a single profile for all

### Fill UDEA_authors with WOS_author info

Obtain UDEA_authors DataFrame: `aunly`

In [118]:
aunly=DataFrame_authors(UDEA)

TABORDA AGUDELO FARLAN


In [119]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [120]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [121]:
UDEA.shape

(15700, 181)

In [122]:
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [123]:
aunly.shape

(1273, 2)

(800, 2)

## Merge UDEA with authors

In [124]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(lambda l:fill_full_wos_author_info(l,aunly) )

In [125]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [126]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [127]:
UDEA.shape

(15700, 181)

In [128]:
kk=UDEA.authors_WOS.combine(UDEA.UDEA_authors,func=lambda x,y: get_UDEA_authors(x,y,aunly))

In [129]:
UDEA.UDEA_authors.dropna().shape

(10312,)

(7072,)

(10960,)

In [130]:
UDEA['UDEA_authors']=kk

In [131]:
UDEA.UDEA_authors.dropna().shape,UDEA.shape

((10900,), (15700, 181))

((8446,), (15700, 169))

((10963,), (15704, 181))

In [132]:
aunly.shape

(1273, 2)

(1461, 2)

In [133]:
if not aunly.empty:
    print(aunly.drop_duplicates('tmp_author').shape)

(1273, 2)


In [134]:
if not aunly.empty:
    aunly.to_json('UDEA_authors_with_WOS_info.json')

In [135]:
RECOVER=False
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [136]:
if UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA.to_json('UDEAtmp.json')

In [137]:
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [138]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [139]:
if 'UDEA_autores' in UDEA.columns:
    print(UDEA[UDEA['UDEA_autores']==''].shape[0],UDEA['UDEA_autores'].dropna().shape[0])

0 10312


In [140]:
if 'UDEA_authors' in UDEA.columns:
    print(UDEA[UDEA['UDEA_authors']==''].shape[0],UDEA['UDEA_authors'].dropna().shape[0])

0 10900


In [141]:
print 1

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-141-c94594b6b28f>, line 1)

## Add PTJ directly from `UDEA_authors` with `WOS_info` DataFrame

In [9]:
UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip').reset_index(drop=True)

In [10]:
aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json').reset_index(drop=True)

In [11]:
def build_institutional_authors(x,author_df,x_author_key='WOS_author',x_affiliation_key='affiliation',
                                        author_key='WOS_author',
                                        affiliation_key='WOS_affiliation'):
    if type(x)!=list:
        return None
    ll=[]
    for j in range(len(x)):
        
                                #author_WOS→affiliation always have single affiliation
        kk=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=0.9 )
        if kk:
            ll.append(kk)
    if not ll:
        ll=None
    return ll

In [12]:
if not UDEA['UDEA_authors'].dropna().shape[0]:
    UDEA['UDEA_authors']=UDEA.authors_WOS.apply(lambda l: build_institutional_authors(l,aunly) )

## Experimental: Change similirity by merge search

In [13]:
UDEA_YES=UDEA[~UDEA['UDEA_authors'].isna()].reset_index(drop=True)

In [14]:
UDEA_YES.shape

(10900, 181)

In [15]:
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
UDEA_NOT=UDEA[UDEA['UDEA_authors'].isna()].reset_index(drop=True)
df2=aunly.copy()
df2=pd.DataFrame( list( df2['UDEA_authors'].values ) )
df2['UDEA_authors']=aunly['UDEA_authors']
contents=df2[['WOS_author','WOS_affiliation','UDEA_authors']].reset_index(drop=True)
contents['WOS_author']=contents['WOS_author'].astype(str)
contents['WOS_affiliation']=contents['WOS_affiliation'].astype(str)

In [16]:
print( UDEA_NOT['authors_WOS'].loc[0][0].get('WOS_author'),
      fwp.extractOne(  UDEA_NOT['authors_WOS'].loc[0][0].get('WOS_author'),
                     contents['WOS_author'],scorer=fuzz.partial_ratio  ) )

Velez-Cuartas, Gabriel ("['Velez Cuartas, Gabriel', 'Velez-Cuartas, Gabriel', 'Velez-Cuartas, G.']", 100, 1037)


In [17]:
dfnot=UDEA_NOT.copy()
dfnot=dfnot.reset_index(drop=True)

In [18]:
#for i in range(20):
#l=dfnot['authors_WOS'].loc[i]
def json_fuzzy_merge(l,contents,right_target='UDEA_authors',
                       left_on='WOS_author',extra_left_on='affiliation',
                       right_on='WOS_author',extra_right_on='WOS_affiliation',
                       cuttoff=95,cuttof_extra=65,scorer=fuzz.partial_ratio):
    newl=[]
    for d in l:
        au=d.get(left_on)
        aff=d.get(extra_left_on)
        # Do not need to be string
        r=fwp.extractOne(au,contents[right_on],scorer=scorer)
        if r[1]>=cuttoff:
            raf=scorer( aff, contents.loc[r[2],extra_right_on]  )
            if raf>=cuttof_extra:
                newl=newl+[  contents.loc[r[2],right_target]  ]
    if newl:
        return newl
    else:
        return None

In [24]:
import pandas as pd
import swifter

In [26]:
%time kk=dfnot['authors_WOS'].swifter.apply(lambda l: json_fuzzy_merge(l,contents,cuttof_extra=65))


CPU times: user 5min 45s, sys: 736 ms, total: 5min 45s
Wall time: 5min 45s


In [27]:
%time kk=dfnot['authors_WOS'].apply(lambda l: json_fuzzy_merge(l,contents,cuttof_extra=65))

CPU times: user 5min 31s, sys: 12 ms, total: 5min 31s
Wall time: 5min 31s


In [28]:
kk.dropna().shape

(781,)

In [29]:
dfnot['UDEA_authors']=kk

In [30]:
#dfnot[['UDEA_authors','authors_WOS']].dropna(subset=['UDEA_authors'])

In [31]:
UDEA_NOT=dfnot.reset_index(drop=True)

In [32]:
UDEA=UDEA_YES.append(UDEA_NOT).reset_index(drop=True)

In [33]:
UDEA['UDEA_authors'].dropna().shape

(11681,)

In [35]:
UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',compression='gzip')

In [ ]:
print 1

## Try other approachs

In [474]:
wp.merge_with_close_matches??

In [477]:
%%writefile test.cfg
[FILES]
Sample_WOS.xlsx = 1--LJZ4mYyQcaJ93xBdbnYj-ZzdjO2Wq2
Sample_SCI.xlsx = 1-3a-hguQTk5ko8JRLCx--EKaslxGVscf
Sample_SCP.xlsx = 1-IAWlMdp2U-9L2jvZUio04ub1Ym3PX-H

Writing test.cfg


In [479]:
cib=wp.wosplus('test.cfg')
#cib.Debug=True
cib.load_biblio('Sample_WOS.xlsx')
cib.load_biblio('Sample_SCI.xlsx',prefix='SCI')
cib.load_biblio('Sample_SCP.xlsx',prefix='SCP')

In [558]:
def get_close_matches_Levenshtein(
        word,
        possibilities,
        n=3,
        cutoff=0.6,
        full=False):
    '''Replaces difflib.get_close_matches with faster algortihm based on
       Levenshtein.ratio.
       HINT: Similarity increase significatively after lower() and unidecode()

       Refs: https://en.wikipedia.org/wiki/Levenshtein_distance
    '''
    import pandas as pd
    import Levenshtein
    if isinstance(possibilities, str):
        possibilities = [possibilities]
    rs = pd.DataFrame()
    MATCH = False
    for p in possibilities:
        similarity = Levenshtein.ratio(word, p)
        # print(word,'::',p,similarity)
        # sys.exit()
        if similarity >= cutoff:
            MATCH = True
            rs = rs.append({'similarity': similarity,
                            'match': p}, ignore_index=True)

    if MATCH:
        rs = rs.sort_values(
            'similarity', ascending=False).reset_index(drop=True)
        if full:
            return list(rs['match'][:n].values), list(
                rs['similarity'][:n].values)
        else:
            return list(rs['match'][:n].values)
    else:
        if full:
            return ([], 0)
        else:
            return []

In [633]:
s='MALL: THE EXAMPLE OF Paracoccidioides brasiliensis CONIDIA'

In [634]:
get_close_matches_Levenshtein(
            s, possibilities, n=1, cutoff=0, full=True)

(['the power of the small: the example of paracoccidioides brasiliensis conidia'],
 [0.5074626865671642])

In [641]:
[c for c in [2,4]]

[2, 4]

In [646]:
r=fwp.extract(s,possibilities,scorer=scorer,limit=2)

In [657]:
cutoff

0.6

In [ ]:
scorer=fuzz.partial_token_set_ratio

In [764]:
def get_close_matches_Levenshtein_new(
        word,
        possibilities,
        n=1,
        cutoff=0.6,
        full=False,
        scorer=fuzz.partial_ratio): #
    r=fwp.extract(word,possibilities,scorer=scorer,limit=n)
    
    if r[0][1]/100.>cutoff:
        if full:
            return [t[0] for t in r],[t[1]/100. for t in r]
        else:
            return [t[0] for t in r]
    else:
        if full:
            return ([], 0)
        else:
            return []        

In [765]:
s='kjdgslln sldg'

In [766]:
get_close_matches_Levenshtein_new(
            s, possibilities, n=2, cutoff=0.6, full=True)

([], 0)

In [767]:
def merge_with_close_matches_old(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.6):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

def merge_with_close_matches_new(
        left,
        right,
        left_on='ST',
        right_on='UDEA_simple_título',
        left_extra_on='SO',
        right_extra_on='UDEA_nombre revista o premio',
        how='inner',
        n=1,
        cutoff=0.6,
        full=True,
        cutoff_extra=0.7):
    '''For each entry of the column: left_on of DataFrame left (cannot have empty fields),
       try to find the close match inside each row of right DataFrame, by comparing with
       the right_on entry of the row. When a row match is found, the full right row is appended
       to the matched row in the left DataFrame.
       If the similarity between the entries at left_on and right_on is less than 0.8,
       an additional check is performed between the entries left_extra_on and right_extra_on
       of the matched row.

       how implemented: inner and left (Default: inner)
    '''
    import numpy as np
    from unidecode import unidecode
    import pandas as pd
    # import sys #import globally
    # print(left[left_on][0])
    # sys.exit()
    words = left[left_on].str.lower().map(unidecode)
    possibilities = right[right_on].str.lower().map(unidecode)

    joined = pd.DataFrame()
    mi = np.array([])
    for i in left.index:
        if i % 100 == 0:
            print('.', end="")
        joined_series = left.loc[i]
        #joined_series=joined_series.append(pd.Series( {similarity_column:0} ))
        title, similarity = get_close_matches_Levenshtein_new(
            words[i], possibilities, n=n, cutoff=cutoff, full=full)
        # print(i,words[i],title,similarity) #cutuff 0.6 0.7 0.8 0.85 0.91 0.95
        # sys.exit()
        if title:
            mtch = right[possibilities == title[0]]
            # >=cutoff, e.g 0.65 0.95 0.81 0.86 0.9 0.96
            chk_cutoff = similarity[0]
            crosscheck = cutoff + 0.2  # 0.8 # e.g. 0.8 0.9 0.9 0.9 0.9 0.9
            if crosscheck >= 1:
                # force check if match worst than this (by experience)
                crosscheck = 0.95
            if chk_cutoff < crosscheck:  # e.g 0.65<0.8 0.95~<0.9 0.81~<0.0 0.86<0.9 0.91<~0.9 0.96~<0.9
                if get_close_matches_Levenshtein_new(unidecode(left[left_extra_on][i].lower()), [unidecode(
                        mtch[right_extra_on][mtch.index[0]].lower())], cutoff=cutoff_extra):  # cutoff=0.6
                    chk_cutoff = crosscheck + 0.1

            if chk_cutoff >= crosscheck:
                joined_series = joined_series.append(mtch.loc[mtch.index[0]])
                if how == 'outer':
                    mi = np.concatenate((mi, mtch.index.values))
                # joined_series[similarity_column]=similarity[0]

            #return joined_series
            if how == 'inner':
                joined = joined.append(joined_series, ignore_index=True)

        if (how == 'left' or 'outer'):
            joined = joined.append(joined_series, ignore_index=True)
    if how == 'outer':
        joined = joined.append(right.drop(
            right.index[list(mi.astype(int))]).reset_index(drop=True))
    return joined

In [802]:
cib.biblio['WOS']=UDEA.sample(500).reset_index(drop=True).copy().fillna('')
cib.biblio['SCI']=SIU[0:100].copy().fillna('')

In [772]:
tmp=drive_files.load_biblio('produccion_fecha_vig_2003_2018.xlsx',prefix='UDEA')

In [773]:
drive_files.biblio['UDEA']

(46212, 20)

In [783]:
s=time.time()
kkold=merge_with_close_matches_old(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),
                            left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')
print(time.time()-s)

.3.9596588611602783


In [784]:
s=time.time()
kknew=merge_with_close_matches_new(cib.biblio['WOS'],cib.biblio['SCI'].drop('Tipo',axis='columns'),
                            left_on='TI',right_on='SCI_TI',right_extra_on='SCI_SO',how='left')
print(time.time()-s)

.3.8951470851898193


In [7]:
kkold.shape,kknew.shape

NameError: name 'kkold' is not defined

In [789]:
(kkold['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['SCI_TI'].apply(lambda s: s if s else pd.np.nan).dropna().shape)

((97,), (97,))

In [6]:
kknew[['TI','SCI_TI']].dropna()

NameError: name 'kknew' is not defined

In [753]:
kkold[['TI','SCI_TI']].dropna()

,TI,SCI_TI
0,The processes involved in a management model for university patents,
1,"Reforma a la salud y reconfiguración de la trayectoria de acceso a los servicios de salud desde la experiencia de los usuarios en Medellín, Colombia","Reforma a la salud y reconfiguración de la trayectoria de acceso a los servicios de salud desde la experiencia de los usuarios en Medellín, Colombia"
2,"Diferencias socioeconómicas en los años de vida perdidos por cáncer de mama y cáncer cérvico-uterino en Colombia, 1997 y 2007","Diferencias socioeconómicas en los años de vida perdidos por cáncer de mama y cáncer cérvico-uterino en Colombia, 1997 y 2007"
3,"Efecto del enriquecimiento con nutrientes en la degradación de clorpirifos, malatión y metil paratión","Efecto del enriquecimiento con nutrientes en la degradación de clorpirifos, malatión y metil paratión"
4,El Balanced Scorecard como sistema de control y su aplicación como herramienta de evaluación en el ambiente bibliotecario,El Balanced Scorecard como sistema de control y su aplicación como herramienta de evaluación en el ambiente bibliotecario
5,"Papel del bibliotecario público en la tradición educativa y cultural de Medellín, 1870 - 1950","Papel del bibliotecario público en la tradición educativa y cultural de Medellín, 1870 - 1950"
6,Reflexividad dialógica en el estudio de jóvenes y prácticas políticas,Reflexividad dialógica en el estudio de jóvenes y prácticas políticas
7,El retorno a la traducción o nuevamente sobre la historia del concepto de hermenéutica,El retorno a la traducción o nuevamente sobre la historia del concepto de hermenéutica
8,An Algebraic Characterization of Affine Manifolds with G-Structure Satisfying a Homogeneity Condition,An Algebraic Characterization of Affine Manifolds with G-Structure Satisfying a Homogeneity Condition
9,"ESTUDIO DEL POLIMORFISMO HINFI DEL GEN PIT-1 Y SU ASOCIACIÓN CON CARACTERÍSTICAS DE TIPO, PRODUCCIÓN DE LECHE Y DÍAS ABIERTOS DE VACAS HOLSTEIN EN EL DEPARTAMENTO DE ANTIOQUIA, COLOMBIA","ESTUDIO DEL POLIMORFISMO HINFI DEL GEN PIT-1 Y SU ASOCIACIÓN CON CARACTERÍSTICAS DE TIPO, PRODUCCIÓN DE LECHE Y DÍAS ABIERTOS DE VACAS HOLSTEIN EN EL DEPARTAMENTO DE ANTIOQUIA, COLOMBIA"


In [803]:
s=time.time()
kkold=merge_with_close_matches_old(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

.....506.1962308883667


In [804]:
s=time.time()
kknew=merge_with_close_matches_new(cib.biblio['WOS'][['TI','SO']],drive_files.biblio['UDEA'][['UDEA_título',
                                                            'UDEA_nombre revista o premio']],
                            left_on='TI',left_extra_on='SO',right_on='UDEA_título',
                            right_extra_on='UDEA_nombre revista o premio',how='left')
print(time.time()-s)

....12330.449292421341


In [805]:
..................

SyntaxError: invalid syntax (<ipython-input-805-a56e8492fbb8>, line 1)

In [806]:
(kkold['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape,
 kknew['UDEA_título'].apply(lambda s: s if s else pd.np.nan).dropna().shape)

((100,), (431,))

In [835]:
kknew[kknew.TI.combine(kknew['UDEA_título'],func=
                       lambda x,y:lv.ratio(x,y) 
                       if type(x)==str and type(y)==str else 0)>0.5
           ].reset_index(drop=True)

,SO,TI,UDEA_nombre revista o premio,UDEA_título
0,CHUNGARA-REVISTA DE ANTROPOLOGIA CHILENA,HIV/AIDS SITUATION AMONG THE WAYUU IN COLOMBIA: SEEKING TO UNDERSTAND THEIR VIEWPOINT,CHUNGARA-REVISTA DE ANTROPOLOGIA CHILENA,CARACTERIZACION DE LA SITUACION DEL VIH/SIDA EN LOS WAYUU DE COLOMBIA: UNA APROXIMACION A SUS PERCEPCIONES. (HIV/AIDS SITUATION AMONG THE WAYUU IN COLOMBIA: SEEKING TO UNDERSTAND THEIR VIEWPOINT).
1,POWDER DIFFRACTION,X-ray powder diffraction data for two new compounds (NH4)(1.5)Ni2V2O7(OH)(1.5) center dot H2O and (NH4)(1.5)Cu1.125Ni1.125V2O7(OH)(2) center dot H2O,POWDER DIFFRACTION,"""X-RAY POWDER DIFFRACTION DATA FOR TWO NEW COMPOUNDS (NH4)1.5Ni2V2O7(OH)1.5 H2O AND (NH4)1.5Cu1.125Ni1.125V2O7(OH)2 H2O"""
2,Revista Colombiana de Antropología,"CUERPOS TATUADOS, ""ALMAS"" TATUADAS: NUEVAS FORMAS DE SUBJETIVIDAD EN LA CONTEMPORANEIDAD",REVISTA COLOMBIANA DE ANTROPOLOGIA,"""CUERPOS TATUADOS ""ALMAS"" TATUADAS: NUEVAS FORMAS DE SUBJETIVIDAD EN LA CONTEMPORANEIDAD"""
3,Análisis Político,LA CONSTRUCCIÓN SOCIAL DE UNA GUERRILLA,ANALISIS POLITICO,LA CONSTRUCCION SOCIAL DE UNA GUERRILLA. (THE SOCIAL CONSTRUCTION OF A GUERRILLA).
4,Biotecnología en el Sector Agropecuario y Agroindustrial,EVALUACIÓN DE LA CALIDAD DE CARAMELOS DE UCHUVA SIN SACAROSA ADICIONADOS CON CALCIO,BIOTECNOLOGIA EN EL SECTOR AGROPECUARIO Y AGROINDUSTRIAL,"""EVALUACION DE LA CALIDAD DE CARAMELOS DE UCHUVA SIN SACAROSA ADICIONADOS CON CALCIO"" (QUALITY ASSESSMENT OF CAPE GOOSEBERRY CANDY WITH ADDED CALCIUM WITHOUT SUCROSE)"
5,Actualidades Biológicas,MICROPROPAGACIÓN E INDUCCIÓN DE ÓRGANOS DE ALMACENAMIENTO EN CURCUMA LONGA L.,ACTUALIDADES BIOLOGICAS,"""MICROPROPAGACION E INDUCCION DE ORGANOS DE ALMACENAMIENTO EN CURCUMA LONGAL L"" (MICROPROPAGATION AND INDUCTION OF STORAGE ORGANS IN CURCUMA LONGA L)"
6,DYNA,MODELADO DE FLUJO COMPRESIBLE UNIDIMENSIONAL Y HOMOENTRÓPICO POR EL METODO DE VOLÚMENES FINITOS,REVISTA DYNA,"""MODELADO DE FLUJO COMPRESIBLE UNIDIMENSIONAL Y HOMOENTROPICO POR EL METODO DE VOLUMENES FINITOS"" (UNIDIMENSIONAL AND HOMOENTROPIC COMPRESIBLE FLOW MODELLING USING THE FINITE VOLUME METHOD)"
7,DYNA,CARACTERIZACIÓN TEXTURAL Y QUÍMICA DE CATALIZADORES TI-SIO2 CRISTALINOS USADOS EN LA EPOXIDACIÓN DE ÉSTERES GRASOS,REVISTA DYNA,"""CARACTERIZACION TEXTURAL Y QUIMICA DE CATALIZADORES TI-SIO2 CRISTALINOS USADOS EN LA EPOXIDACION DE ESTERES GRASOS"" (TEXTURAL AND CHEMICAL CHARACTERIZATION OF CRYSTALLINE TI-SIO2 CATALYSTS USED I..."
8,Acta Biológica Colombiana,"CYP1A1, CYP2E1 Y RIESGO A CÁNCER GÁSTRICO EN UNA POBLACIÓN COLOMBIANA DE ALTA INCIDENCIA",ACTA BIOLOGICA COLOMBIANA,"""CYP1A1, CYP2E1 Y RIESGO A CANCER GASTRICO EN UNA POBLACION COLOMBIANA DE ALTA INCIDENCIA"" (CYP1A1, CYP2E1 AND GASTRIC CANCER RISK IN A HIGH-INCIDENDE COLOMBIAN POPULATION)"
9,Cuadernos de Economía,COMPENSACIONES ECONÓMICAS ANTE CONFLICTOS DE USO DEL SUELO,CUADERNOS DE ECONOMIA,"""COMPENSACIONES ECONOMICAS ANTE CONFLICTOS DE USO DEL SUELO"""


In [837]:
kkold[[TI,]]

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_tipo mat,UDEA_tipo mov,UDEA_título,UDEA_título_list,UDEA_valor item,UT,VL,WC,Z9,authors_WOS
0,"Current tendencies in strengthening intellectual property rights and the emergence of new scientific-technological fields (such as nanotechnologies, biotechnology, telematics and software developm...","Lopez Gomez, Maria del Socorro\nSchmal Simon, Rodolfo\nCabrales Gomez, Fernando\nGarcia Tobon, Carolina\n",,"Gomez, MDL\nSimon, RS\nGomez, FC\nTobon, CG\n",,,,,135\n,"[Lopez Gomez, Maria del Socorro] Univ Antioquia, Fac Ciencias Econ, Bogota, Colombia.\n[Schmal Simon, Rodolfo] Univ Talca, Fac Ciencias Empresariales, Talc, Chile.\n[Cabrales Gomez, Fernando] Univ...",...,ARTREVA1,NO,"""LOS PROCESOS EN UN MODELO DE GESTION DE PATENTES UNIVERSITARIAS"" (THE PROCESSES INVOLVED IN A MANAGEMENT MODEL FOR UNIVERSITY PATENTS)","[LOS PROCESOS EN UN MODELO DE GESTION DE PATENTES UNIVERSITARIAS, THE PROCESSES INVOLVED IN A MANAGEMENT MODEL FOR UNIVERSITY PATENTS]",,WOS:000271839000022\n,29\n,"Engineering, Multidisciplinary\n",0.0,"[{'WOS_author': 'Lopez Gomez, Maria del Socorro', 'affiliation': ['Univ Antioquia, Fac Ciencias Econ, Bogota, Colombia.'], 'i': 0}]"
1,,,,"Echeverry-López, María Esperanza\n",,,,,,"[Echeverry-López, María Esperanza] Univ Antioquia, Colombia.\n",...,,,"REFORMA A LA SALUD Y RECONFIGURACION DE LA TRAYECTORIA DE ACCESO A LOS SERVICIOS DE SALUD DESDE LA EXPERIENCIA DE LOS USUARIOS EN MEDELLIN, COLOMBIA (HEALTH REFORM AND RECONFIGURATION OF THE TRAJE...",,,,,,0.0,"[{'WOS_author': 'Echeverry-López, María Esperanza', 'affiliation': ['Univ Antioquia, Colombia.'], 'i': 0}]"
2,,,,"Agudelo-Botero, Marcela\nDávila-Cervantes, Claudio Alberto\nAtehortúa-Becerra, Sara Catalina\n",,,,,,"El Colegio de México, Mexico.\nUniv Antioquia, Colombia.\nEl Colegio de México, Mexico.\n",...,,,"DIFERENCIAS SOCIOECONOMICAS EN LOS ANOS DE VIDA PERDIDOS POR CANCER DE MAMA Y CANCER CERVICO-UTERINO EN COLOMBIA, 1997 Y 2007 (SOCIOECONOMIC DIFFERENCES IN THE YEARS OF LIFE LOST DUE TO BREAST CAN...",,,,,,0.0,[]
3,,,,"Upegui, S. A.\nPeñuela, G. A.\nBotero, L. R.\n",,,,,,"[Peñuela, G. A.] Univ Antioquia, Colombia.\n[Botero, L. R.] Universidad de Medellín, Colombia.\n[Upegui, S. A.] Universidad de Medellín, Colombia.\n",...,,,"EFECTO DEL ENRIQUECIMIENTO CON NUTRIENTES EN LA DEGRADACION DE CLORPIRIFOS, MALATION Y MENTIL PARATION (INORGANIC NUTRIENTS EFFECT ADDITION OVER THE CHROLPYRIFOS, MALATHION AND METHYL PARATHION DE...",,,,,,0.0,"[{'WOS_author': 'Peñuela, G. A.', 'affiliation': ['Univ Antioquia, Colombia.'], 'i': 0}]"
4,,,,"Montoya Agudelo, César Alveiro\nBárbaro, Juan Carlos\n",,,,,,"[Bárbaro, Juan Carlos] Universidad Nacional de Misiones, Argentina.\n[Montoya Agudelo, César Alveiro] Univ Antioquia, Colombia.\n",...,,,EL BALANCED SCORECARD COMO SISTEMA DE CONTROL Y SU APLICACION COMO HERRAMIENTA DE EVALUACION EN EL AMBIENTE BIBLIOTECARIO,,,,,,0.0,"[{'WOS_author': 'Montoya Agudelo, César Alveiro', 'affiliation': ['Univ Antioquia, Colombia.'], 'i': 0}]"
5,,,,"Montoya Ríos, Mónica\nGiraldo Lopera, Marta Lucía\nGallo Vélez, óscar\nMoncada Patiño, José Daniel\n",,,,,,"[Giraldo Lopera, Marta Lucía] Univ Antioquia, Colombia.\n[Gallo Vélez, óscar] Universidad Nacional de Colombia, Colombia.\n[Montoya Ríos, Mónica] Univ Antioquia, Colombia.\n[Moncada Patiño, José D...",...,,,"PAPEL DEL BIBLIOTECARIO PUBLICO EN LA TRADICION EDUCATIVA Y CULTURAL DE MEDELLIN, 1870-1950",,,,,,0.0,"[{'WOS_author': 'Giraldo Lopera, Marta Lucía', 'affiliation': ['Univ Antioquia, Colombia.'], 'i': 0}, {'WOS_author': 'Montoya Ríos, Mónica', 'affiliation': ['Univ Antioquia, Colombia.'], 'i': 1}, ..."
6,,,,"Ghiso, Alfredo\nTabares-Ochoa, Catalina María\n",,,,,,"[Tabares-Ochoa, Catalina María] Univ Antioquia, Colombia.\n[Ghiso, Alfredo] Fundación Universitaria Luis Amigó, Colombia.\n",...,,,REFLEXIVIDAD DIALOGICA EN EL ESTUDIO DE JOVENES Y PRACTICAS POLITICAS (DIALOGIC REFLEXIVITY IN THE STUDY OFYOUTH AND POLITICAL PRACTICES),,,,,,0.0,"[{'WOS_author': '

In [816]:
fuzz.partial_token_set_ratio(kknew.TI.loc[3],'nausea')

100

In [817]:
fuzz.ratio(kknew.TI.loc[3],'nausea')

3

In [818]:
fuzz.ratio('z','z')

100

In [813]:
lv.ratio(kknew.TI.loc[3],'z')

0.005780346820809248